# Detecting the OpenMaker Opinion Leaders

In [ ]:
from LibOM.Tools import *

### 1. Retrieving influencers from the Watch Tower API
The object that connects to [the Watch Tower](http://138.68.92.181:8484/api) API in order to retrieve the list of top influencers under each technical categories such as Arduino, 3D Printing and Raspberry Pi.

In [ ]:
ClientWT = WatchTower()
influencer_names = ClientWT.retrieve_influencers()

In [ ]:
print(len(influencer_names))
print("_"*25)
for inf in influencer_names: print(inf)

### 2. Connecting to the Twitter API
In order to get credentials to be able to connect to the Twitter:
- Login to https://apps.twitter.com 
 - If you already have twitter application you may use its credentials.
 - Or generate a new set of keys and tokens via "Create a New App" button.
- Click "Keys and Access Tokens" tab and copy "API key", API secret".
- Click "Create my Access Token" and copy "Access token" and "Access token secret".

In [ ]:
Credentials = {}
Credentials['Consumer_Key'] = "your key" 
Credentials['Consumer_Secret'] = "your secret"
Credentials['Access_Token'] = "your token" 
Credentials['Access_Token_Secret'] = "your token secret"

In [ ]:
T = Twitter(Credentials)

In [ ]:
debates = T.retrieve_tweets(influencer_names, nlatest = 10)

In [ ]:
for user,data in debates.items():
    print("_"*80)
    print("Username: {}".format(user))
    print("Number of tweets: {}".format(data['ntweets']))
    print("-"*20)
    for tweet in data['content'].split("\n"): print(tweet)

### 3. Internal dictionary based text annotaion/feature extraction
This module will be revised. It will be the major focus of this work in further iterations. The current mapping from expressions to categories are tentative. For the time being it is mainly for a demonstrative purpose. 

In [ ]:
MD = MakerDictionary()

In [ ]:
for name, code in MD.categories.items(): print name, code

In [ ]:
# The tentative list of expressions under each category:
for category, mappings in MD.table_Mfeatures.items():
    print "_"*60
    print MD.get_category_name(category), " :: ", category
    print sorted(mappings['content'])

#### Mapping expressions to categories:

In [ ]:
SB = ScoreBoard()
for user in debates.keys():
    ntweets = debates[user]['ntweets']
    text = debates[user]['content']
    nmappings, nwords, counts = extract_features(text, MD)
    features = {"ntweets":ntweets, "nwords":nwords, "nmappings":nmappings, "counts":counts}
    SB.add_actor(user, features)
    print user, ntweets, nwords, nmappings
    print counts

In [ ]:
for k, v in SB.table.items():
    print "_" * 20
    print k, v["ntweets"], v["nwords"], v["nmappings"]
    for type in v['scores'].keys():
        print type, v['scores'][type]

### 4. Computing the rankings

In [ ]:
SB.compute_rankings('0', 'per_word')
SB.compute_rankings('5', 'per_word')
SB.compute_rankings('all', 'per_word')

In [ ]:
def print_score_table(rankings):
    for rtype, scores in rankings.items():
        print "_"*20
        print rtype
        for user,score in list(scores): print user, " : ", score

In [ ]:
print_score_table(SB.rankings)

In [ ]:
SB.get_rankings_one('7', 'per_word')
SB.get_rankings_one('1', 'raw')
SB.get_rankings_one('all', 'per_tweet')
print_score_table(SB.rankings)

#### Getting overall Makership score of an influencer:

In [ ]:
username = '3DPrintGirl'
score = SB.get_score_one(username, 'all', 'per_tweet')
print score

#### Getting components of the overall score over mapped categories:

In [ ]:
cat_codes = MD.categories.values()
sub_scores = SB.get_scores(username, cat_codes, 'per_tweet')
print sub_scores

### 5. Computing the makership scores of a given tweeter user and placing it on the scoreboard

In [ ]:
def get_anew_user_score(username):
    tdata = ClientTwitter.accumulate_auser_tweets(username)
    ntweets = tdata['ntweets']
    if not ntweets: return  
    text = tdata['content']
    nmappings, nwords, counts = extract_features(text, MD)
    features = {"ntweets":ntweets, "nwords":nwords, "nmappings":nmappings, "counts":counts}
    SB.add_actor(username, features)
    score = SB.get_score_one(username, 'all', 'per_tweet')
    categories = MD.categories.values()
    sub_scores = SB.get_scores(username, categories, 'per_tweet')
    return ntweets, sub_scores

In [ ]:
for score in SB.get_rankings_one('all', 'per_tweet'): print score

In [ ]:
tweetdata = get_anew_user_score('BernieSanders')
print "Number of tweets used: ", tweetdata[0]
print "scores (theme:score)::"
for theme,score in tweetdata[1].items():
    print theme, ":", score

In [ ]:
print SB.get_score_one('BernieSanders', 'all', 'per_tweet')

In [ ]:
for score in SB.get_rankings_one('all', 'per_tweet'): print score